# Load Horizons Data into KeplerDB

In [9]:
# Core
import numpy as np
import pandas as pd

# Utility
import re
import glob
import time

# Database
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
# MSE imports
import kepler_sieve
from horizons_files import hrzn_txt2csv, hrzn_csv2df, hrzn_df2db, hrzn_txt2db, hrzn_load
import db_config

In [3]:
# fname_txt = '../data/jpl/horizons/planets/001_mercury_barycenter.txt'
# fname_txt = '../data/jpl/horizons/planets/010_sun.txt'
fname_txt = '../data/jpl/horizons/asteroids/ast_0001.txt'

In [ ]:
with open(fname_txt) as fh:
    lines = fh.readlines()

In [ ]:
for line in lines:
    if line[0:18] == 'Target body name: ':
        target_body = line[18:]
        break

In [ ]:
target_body

In [ ]:
# regex search pattern
# p = re.compile('(^.+) \(([\d]+)\)[\s]+\{source: (.+)\}\n$')
p = re.compile('(^[0-9]+) (.+) \(.*\)[\s]+\{source: (.+)\}\n$')

In [ ]:
# extract name, number, and source from the target body string
srch = p.search(target_body)
print(srch.group(1))
print(srch.group(2))
print(srch.group(3))
# body_name = srch.group(1)
# body_number = srch.group(2)
# integration_source = srch.group(3)

In [ ]:
'ast_' in fname_txt

In [ ]:
srch.group(3)

In [4]:
fname_csv = hrzn_txt2csv(fname_txt)

In [5]:
df = hrzn_csv2df(fname_csv)

In [6]:
pd.set_option('display.max_rows', 20)

In [7]:
df

,BodyNumber,BodyName,IntegrationSource,JD,CalendarDateTime,delta_T,qx,qy,qz,vx,vy,vz
0,1,Ceres,JPL#47,2440400.5,1969-06-28,39.699541,1.443184,-2.549274,-0.340158,0.008465,0.004489,-0.001436
1,1,Ceres,JPL#47,2440405.5,1969-07-03,39.712359,1.485300,-2.526463,-0.347286,0.008381,0.004635,-0.001416
2,1,Ceres,JPL#47,2440410.5,1969-07-08,39.725179,1.526988,-2.502924,-0.354315,0.008294,0.004780,-0.001396
3,1,Ceres,JPL#47,2440415.5,1969-07-13,39.738000,1.568239,-2.478664,-0.361242,0.008206,0.004923,-0.001375
4,1,Ceres,JPL#47,2440420.5,1969-07-18,39.750824,1.609040,-2.453693,-0.368064,0.008115,0.005065,-0.001354
5,1,Ceres,JPL#47,2440425.5,1969-07-23,39.763650,1.649382,-2.428017,-0.374782,0.008022,0.005205,-0.001333
6,1,Ceres,JPL#47,2440430.5,1969-07-28,39.776478,1.689253,-2.401648,-0.381392,0.007926,0.005343,-0.001311
7,1,Ceres,JPL#47,2440435.5,1969-08-02,39.789310,1.728642,-2.374592,-0.387894,0.007829,0.005479,-0.001289
8,1,Ceres,JPL#47,2440440.5,1969-08-07,39.802149,1.767541,-2.346859,-0.394284,0.007730,0.005614,-0.001267
9,1,Ceres,JPL#47,2440445.5,1969-08-12,39.814996,1.805937,-2.318458,-0.400563,0.007628,0.005746,-0.001244


In [8]:
# hrzn_load()

In [11]:
connection_str = f'mysql+pymysql://{db_config.username}:{db_config.password}@{db_config.hostname}/JPL'

In [12]:
engine = create_engine(connection_str)

In [13]:
engine.table_names()

['HorizonsImport']

In [17]:
t0 = time.time()
with engine.connect() as conn:
    df.to_sql(name='HorizonsImport', con=conn, if_exists='append', index=False)
t1 = time.time()

In [18]:
t1 - t0

1.509110927581787

In [ ]:
hrzn_txt2db(fname_txt)